In [ ]:
import pandas as pd
import numpy as np
from sympy import *
import openai
import time
import random

In [ ]:
# assign variable value with your OpenAI key to continue
openai.api_key = ''

In [ ]:
# 3
def generate_3(s):
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=s,
        temperature=0.7)
    return response['choices'][0]['text']

In [ ]:
# 3.5
def generate_3_5(s):
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[{
            "role": 'user',
            "content": s
        }],
        temperature=0.7
    )
    return completion.choices[0].message.content

In [ ]:
# 4
def generate_4(s):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{
            "role": 'user',
            "content": s
        }],
        temperature=0.7)
    return response.choices[0].message.content

# Within-topic

In [ ]:
# Generate questions based on those in L200
df200 = pd.read_excel('Data/L200.xlsx', header=None)
df200 = df200.rename(columns={0: "Question", 1: "Answer", 2: "Category"})
ans = pd.DataFrame(columns=['Question', 'Category', 'Topic', 'Model3', 'ModelTurbo', 'Model4'])

In [ ]:
# randomly draw sample questions
qs = pd.DataFrame(columns=['Question', 'Category', 'Topic'])
gp200 = df200.groupby('Category')
for name, group in gp200:
    temp = group.reset_index()
    idxs = random.sample(range(0,40),3)
    for i in idxs:
        qs.loc[len(qs)] = [temp.loc[i, 'Question'], name, '']
#qs.to_excel("WTQ.xlsx",index=False)

In [ ]:
# or use what we got
qs = pd.read_excel("WTQ.xlsx")

In [ ]:
# GPT-3
for index, row in qs.iterrows():
    for i in range(0,10):
        s = row['Question']+'Given the above question, use different values to write a similar question \
        about '+row['Topic']+'.'
        if i ==0:
            ans.loc[len(ans)] = [row['Question'], row['Category'], row['Topic'], 
                                 generate_3(s), '', '']
        else:
            ans.loc[len(ans)] = ['', '', '', generate_3(s), '', '']
        time.sleep(0.1)

In [ ]:
# GPT-3.5
for index, row in qs.iterrows():
    for i in range(0,10):
        s = row['Question']+'Given the above question, use different values to write a similar question \
        statement about '+row['Topic']+'. Don\'t try to solve problem. Don\'t repeat the original statement.'
        ans.loc[len(ans)] = ['', '', '', '', generate_3_5(s), '']
        time.sleep(0.2)

In [ ]:
# GPT-4
for index, row in qs.iterrows():
    for i in range(0,10):
        s = row['Question']+'Given the above question, use different values to write a similar question \
        statement about '+row['Topic']+'. Don\'t try to solve problem. Don\'t repeat the original statement.'
        ans.loc[index, 'Model4'] = generate_4(s)
        index += 1

In [ ]:
ans.to_excel('WTG.xlsx', index=False)

# Cross-topic

In [ ]:
# randomly draw questions from L200 as within-topic generation, but need to manually add a column of reasonable new topics
# or use ours
ct = pd.read_excel("CTQ.xlsx")
gens = pd.DataFrame(columns=['Question', 'Category', 'Topic', 'NewTopic', 'Model3', 'ModelTurbo', 'Model4'])

In [ ]:
# GPT-3
for index, row in ct.iterrows():
    for i in range(0,10):
        s = row['Question']+'Given the above question, use different values to write a similar question \
        about '+row['NewTopic']+'. Don\'t try to solve problem. Don\'t repeat the original statement. \
        The new question should still pertain to a system of two equations.'
        if i ==0:
            gens.loc[len(gens)] = [row['Question'], row['Category'], row['Topic'], row['NewTopic'],
                                generate_3(s), '', '']
        else:
            gens.loc[len(gens)] = ['', '', '', '', generate_3(s), '', '']
        time.sleep(0.2)

In [ ]:
# GPT-3.5
for index, row in ct.iterrows():
    for i in range(0,10):
        s = row['Question']+'Given the above question, use different values to write a similar question \
        about '+row['NewTopic']+'. Don\'t try to solve problem. Don\'t repeat the original statement. \
        The new question should still pertain to a system of two equations.'
        if i ==0:
            gens.loc[len(gens)] = [row['Question'], row['Category'], row['Topic'], row['NewTopic'], '',
                                generate_3_5(s), '']
        else:
            gens.loc[len(gens)] = ['', '', '', '', '', generate_3_5(s), '']
        time.sleep(0.2)

In [ ]:
# GPT-4
for index, row in ct.iterrows():
    for i in range(0,10):
        s = row['Question']+'Given the above question, use different values to write a similar question \
        about '+row['NewTopic']+'. Don\'t try to solve problem. Don\'t repeat the original statement. \
        The new question should still pertain to a system of two equations.'
        if i ==0:
            gens.loc[len(gens)] = [row['Question'], row['Category'], row['Topic'], row['NewTopic'], '', '', generate_4(s)]
        else:
            gens.loc[len(gens)] = ['', '', '', '', '', '', generate_4(s)]
        index += 1

In [ ]:
#gens.to_excel('CTG.xlsx', index=False)